In [87]:
from __future__ import print_function
import numpy as np
import random
import sys
import h5py
import os
import random
import argparse

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM

In [88]:
model = model_from_json(open('model.txt').read())
model.load_weights('model.h5')

In [89]:
text = open('trump.txt').read().lower()
chars = set(text)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [90]:
maxlen = 20
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


In [91]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

def random_line(afile):
    lines = open(afile).read().lower().splitlines()
    myline = random.choice(lines)
    return myline


In [114]:
randline = random_line('trump.txt')
for diversity in [0.2, 0.5, 0.8, 1.0]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    sentence = randline[0:20]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for iteration in range(120):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

    print(generated[20:])
    


----- diversity: 0.2
----- Generating with seed: "some people dream of"
some people dream of the world when he has been to the best of the deal. congrats on the trump tower at 10 pm. at the state of the world is 

----- diversity: 0.5
----- Generating with seed: "some people dream of"
some people dream of an embarrassie”
strength person who knows how to see the way they do nothing to soon. i want to see you all the new y

----- diversity: 0.8
----- Generating with seed: "some people dream of"
some people dream of the place fans guy done! if you said novine the president obama can be a luxure to have a chicago ring in the white hou

----- diversity: 1.0
----- Generating with seed: "some people dream of"
some people dream of every thoughts and had what malk poax, who said many defense “your hotels have energy according.” every speech you 
